# DEBS 2022 Grand Challenge 
## Real-time Market Analytics for Rapid Deployment and High Accessiblity using PySpark and Docker

Group 4: Suyeon Wang, JaeKyeong Kim, Sejin Chun

This code is also available at [github](https://github.com/developsu/debs2022gc)

Before executing codes,
- upload the **challenger.proto** file 
- login with your google account 
- give your authorization to google drive for drive mount
- set up your token key
- set up the **folder dir** to store streams of csv files retrieved from challenger system
- Last, move **Runtime**-> Click [**Run all**] (Ctrl + F9)


**Dataset License.** Sebastian Frischbier, Jawad Tahir, Christoph Doblander, Arne Hormann, Ruben Mayer. 
The DEBS 2021 grand challenge: Detecting Trading Trends in Real Financial Tick Data. 
Proceedings of the 16th ACM International Conference on Distributed and Event-based Systems (DEBS'22) 

In [ ]:
TOKEN_KEY = 'sdpiwsqcjhlimgruqdhyckrgwjwlawvh'

## Installation

In [ ]:
!apt-get install openjdk-8-jdk-headless # jdk

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jre-headless amd64 8u312-b07-0ubuntu1~18.04 [28.2 MB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jdk-headless amd64 8u312-b07-0ubuntu1~18.04 [8,298 kB]
Fetched 36.5 MB in 5s (7,098 kB/s)
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading databas

downloads and unzips the apache spark 

In [ ]:
!wget -q -nc https://mirror.navercorp.com/apache/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar -xvf spark-3.1.2-bin-hadoop3.2.tgz

spark-3.1.2-bin-hadoop3.2/
spark-3.1.2-bin-hadoop3.2/R/
spark-3.1.2-bin-hadoop3.2/R/lib/
spark-3.1.2-bin-hadoop3.2/R/lib/sparkr.zip
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/worker/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/worker/worker.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/worker/daemon.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/tests/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/tests/testthat/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/tests/testthat/test_basic.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/profile/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/profile/shell.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/profile/general.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/sparkr-vignettes.html
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/sparkr-vignettes.Rmd
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/sparkr-vignettes.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/index.html
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/R/
spark-3.1.2-

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [ ]:
!pip install findspark

In [ ]:
import findspark
findspark.init()

mounts google drive for the usage of cloud storage

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


installs gRPC library in Python

In [ ]:
!pip install grpcio

setting up libraries

In [ ]:
import pyspark
import pandas
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql import *
from tqdm import tqdm
import ast

import sys
import os
import time
from datetime import datetime
from google.protobuf.timestamp_pb2 import Timestamp
import csv
import pandas as pd
# from dependencies.spark import start_spark
from pyspark.sql.functions import pandas_udf, PandasUDFType
# If grpc is missing: pip install grpcio
import grpc
from google.protobuf import empty_pb2
# If the classes below are missing, generate them:
# You need to install the grpcio-tools to generate the stubs: pip install grpcio-tools
# python -m grpc_tools.protoc -I . --python_out=. --grpc_python_out=. challenger.proto

In [ ]:
!python -m grpc_tools.protoc -I . --python_out=. --grpc_python_out=. challenger.proto

In [ ]:
import challenger_pb2 as ch
import challenger_pb2_grpc as api

## Core functions

This function reads the stream of csv files from remote benchmark system

In [ ]:
def read_stream(folderDir):
    """
    reads Streaming data and sets up the schema 
    :param spark session
    :return: data stream
    """
    global spark
    # print(f"read csv file in {folderDir}")
    stream_df = spark.readStream.format("csv") \
        .option("header", "true") \
        .schema(get_schema()) \
        .load(path=folderDir)
    
    return stream_df

This function represents the schema of stock market data

In [ ]:
def get_schema():
    """
    return schema for DEBS 2022
    :return: StructType
    """
    return StructType([
        StructField("id", LongType(), True),
        StructField("symbol", StringType(), True),
        StructField("Sectype", StringType(), True),
        StructField("lasttradeprice", DoubleType(), True),
        StructField("lastTrade", TimestampType(), True)
    ])

The following functions calculate EMA38 and EMA100 indicators, respectively.

In [ ]:
'''def EMA38_cal(Last_Last):
    global prev_val
    ema38 = Last_Last * 2 / 39 + prev_val * (1 - 2 / 39)
    prev_val = ema38
    return ema38'''

'def EMA38_cal(Last_Last):\n    global prev_val\n    ema38 = Last_Last * 2 / 39 + prev_val * (1 - 2 / 39)\n    prev_val = ema38\n    return ema38'

In [ ]:
# # ema38계산
# prev_EMA38_broadcast = spark.sparkContext.broadcast(prev_EMA38)
# # def EMA38_cal_broad(prev_EMA38_broad):
# def EMA38_cal(symbol, aprice):
#     global prev_EMA38_broadcast
#     '''if symbol not in prev_EMA38_broadcast.value[symbol]:
#       prev_EMA38_broadcast[symbol] = 0'''
#     ema38 = price * 2 / 39 + prev_EMA38_broadcast.value.get(symbol) * (1 - 2 / 39)
#     prev_EMA38_broadcast.value[symbol] = ema38
#     return ema38

In [ ]:
def EMA100_cal(Last_Last):
    global prev_val
    ema100 = Last_Last * 2 / 101 + prev_val * (1 - 2 / 101)
    prev_val = ema100
    return ema100

### Calc EMA

In [ ]:
def calcEMA_func(symbol, lastTradePrice, prev_ema, days):
  # print(days)
  # list_ema = ast.literal_eval(list_ema)
  # match_id = f"{symbol}_{days}"
  # if match_id in list_ema:
  #   prev_EMA = list_ema[match_id]
  # else: # if not symbol in list_ema
  #   prev_EMA = 1.0
  # # EQUATION
  # print(lastTradePrice * (2 / (days + 1)))
  # print(prev_ema  * (1 - 2 / (days + 1)))
  current_EMA = ( lastTradePrice * (2 / (days + 1)) ) + ( prev_ema  * (1 - 2 / (days + 1)) )
  # print(f"calculator ema{days} = (", lastTradePrice,f" * 2/{days+1} + (", prev_ema, f"*(1 - 2 / ({days + 1}) = ", current_EMA)
  #print(current_EMA)
  return current_EMA

In [ ]:
def get_prev_ema_func(s, days):
  # print(s)
  global spark

  df_prevEMA =  spark.sql("select * from global_temp.prevEMA").select("psymbol", "pEMA38", "pEMA100").filter(col("psymbol") == s )
  
  # if days == 38:
  #   df_result = df_prevEMA.where(df_prevEMA.psymbol == s).toPandas()
  # else:
  #   df_result = df_prevEMA.where(df_prevEMA.psymbol == s).toPandas()

  return 1.0

This function determines the choice of buy, sell, or stay(=hold).

In [ ]:
'''def EMA_Comp(EMA38_MINUS_EMA100):
    global prev_val
    minus_now = EMA38_MINUS_EMA100
    if minus_now * prev_val >= 0:
        prev_val = minus_now
        return "Stay"
    else:
        if minus_now > prev_val:
            prev_val = minus_now
            return "Buy"
        else:
            prev_val = minus_now
            return "Sell"'''

'def EMA_Comp(EMA38_MINUS_EMA100):\n    global prev_val\n    minus_now = EMA38_MINUS_EMA100\n    if minus_now * prev_val >= 0:\n        prev_val = minus_now\n        return "Stay"\n    else:\n        if minus_now > prev_val:\n            prev_val = minus_now\n            return "Buy"\n        else:\n            prev_val = minus_now\n            return "Sell"'

### EMA_Comp

In [ ]:
def EMA_Comp(nEMA38_MINUS_EMA100, pEMA38_MINUS_EMA100):
    '''print('nEMA38_MINUS_EMA100 = ', nEMA38_MINUS_EMA100)
    print('pEMA38_MINUS_EMA100 = ', pEMA38_MINUS_EMA100)
    print("곱하기: ", nEMA38_MINUS_EMA100 * pEMA38_MINUS_EMA100)'''
    minus_prev = pEMA38_MINUS_EMA100
    minus_now = nEMA38_MINUS_EMA100
    if minus_now * minus_prev >= 0:
      return "Stay"
    else:
      if minus_now > minus_prev:
          return "Buy"
      else:
          return "Sell"

## Streaming query 1

In [ ]:
# 쿼리1 진행
def Query1(stream):
    # 주어진 각 ID들에 대해 batch 별로 최신의 EMA38과 EMA100을 출력한다. (tail 써보고 안되면 index 만들어서 desc로 뒤집고 그 결과를 다시 sort reverse 로 뒤집음)
    # batch 별로 출력하려면 Structured streaming 이 아닌 Dstream 을 사용해야 함. rdd로 바꾸어 사용해야 하므로 pyspark는 속도가 조금 느릴 수 있음

    global prev_val, spark, stream_df1, prevEMA
    # global initialStateRDD
    
    # stream_preprocessed = stream_preprocessed.select("Sectype", "lastTrade", "lasttradeprice", "symbol")
    # pre_df = stream_preprocessed
    # 5분 길이의 window 당 종가값(Last)만 사용한다고 가정하면 아래와 같이 출력 가능함. 뒤에 orderBy를 해 시간 순서별로 정렬함
    stream_df = stream.select("symbol", "Sectype", "lastTrade", "lasttradeprice").withWatermark("lastTrade", "5 minutes") \
        .groupBy(window("lastTrade", "5 minutes", "5 minutes"), col("symbol")) \
        .agg(lit(last("lasttradeprice")).alias("Last_Last"), lit(last("Sectype")).alias("Sectype_ext"))\
        .orderBy("window")
    # Last_Last 는 double 형 데이터타입.
    # stream_df1 = stream_df => 성공
    # column name 변경

    #stream_df = stream_df.withColumnRenamed("symbol_ext", "symbol")
    stream_df = stream_df.withColumnRenamed("Sectype_ext", "Sectype")
    stream_df = stream_df.withColumnRenamed("Last_Last", "lasttradeprice")

    stream_df = stream_df.withColumn("lastWindowTime", to_timestamp(stream_df.window.end, "yyyy-MM-dd HH:mm:ss"))

    # stream_df1 = stream_df => 성공
    # stream_df.printSchema()
    # window의 start 와 end 부분 이렇게 따로 추출 가능함. 아래의 select의 항목에 "window" 를 넣으면 window로 보임
    # stream_df = stream_df.select("lastWindowTime", "symbol", "lasttradeprice", "Sectype")

    # udf 사용해 dataframe에 EMA38 적용하는 부분
    # spark.udf.register("Last_udf_ema38", EMA38_cal, DoubleType())

    # Last_udf_ema38 = udf(EMA38_cal, DoubleType())
    # stream_df = stream_df.withColumn("EMA38", Last_udf_ema38(stream_df["lasttradeprice"]))

    calcEMA = udf(calcEMA_func, DoubleType())
    findEMA = udf(get_prev_ema_func, DoubleType())
    # stream_df = stream_df.withColumn("EMA38", Last_udf_ema38(stream_df["lasttradeprice"]))

    # join_df = stream_df
    #          .withColumn("EMA38_c", calcEMA(stream_df["symbol"], stream_df["lasttradeprice"], prev_ema['pEMA38'], lit(38)) ) \
    #          .withColumn("EMA100_c", calcEMA(stream_df["symbol"], stream_df["lasttradeprice"], prev_ema['pEMA100'], lit(100)) )

    # print(join_df.toPandas())
    stream_df = stream_df.withColumn("prevEMA38", lit(1.0) )
    stream_df = stream_df.withColumn("prevEMA100", lit(1.0) )

    stream_df = stream_df.withColumn("EMA38", calcEMA(stream_df["symbol"], stream_df["lasttradeprice"], stream_df["prevEMA38"],  lit(38) ) )
    stream_df = stream_df.withColumn("EMA100", calcEMA(stream_df["symbol"], stream_df["lasttradeprice"],stream_df["prevEMA100"], lit(100) ) )

    # calcEMA = udf(calcEMA_func, DoubleType())
    # stream_df = stream_df.withColumn("EMA38", Last_udf_ema38(stream_df["lasttradeprice"]))
    # lag_df = stream_df.withColumn("prevEMA38", lag(stream_df["initEMA38"]).over(Window.partitionBy("symbol").orderBy("lastWindowTime")))
    # stream_df = stream_df.withColumn("EMA100", calcEMA(stream_df["symbol"], stream_df["lasttradeprice"], lit(str(prevEMA.value)), lit(100)) )
    
    # stream_df = stream_df.select("lasttradeprice", "symbol", "lastWindowTime", "EMA38", "Sectype")

    # udf 사용해 dataframe에 EMA100 적용하는 부분
    # Last_udf_ema100 = udf(EMA100_cal, DoubleType())
    # stream_df = stream_df.withColumn("EMA100", Last_udf_ema100(stream_df["lasttradeprice"]))
    # stream_df1 = stream_df.select("symbol", "lasttradeprice","EMA38", "EMA100", "lastWindowTime", "Sectype")
    stream_df = stream_df.select("symbol", "EMA38", "EMA100", "lasttradeprice", "lastWindowTime", "Sectype")

    # 마지막 값들 이런 식으로 추출해서 사용하면 됨
    # stream_preprocessed_last3 = stream_preprocessed.tail(3)
    # 이렇게 할 경우 stream_preprocessed_last3를 .rdd.collect() 로 데이터 형태를 바꿔서 필요한 부분만 발췌해 list 로 변환하기
    # 그 후 아래의 return 에 반환할 list 추가하면 된다.
    # stream_df1 = stream_df
    # prev_val 값 1.0으로 초기화
    # prev_val = 1.0
    
    return stream_df

## Streaming query 2

In [ ]:
# 쿼리2 진행
def Query2(stream_Query1):
    # 주어진 각 ID들에 대해 batch 별로 최신의 세 crossovers 를 찾아 Buy/Sell Advice를 timestamp와 함께 출력한다. (tail 써보고 안되면 index 만들어서 desc로 뒤집고 그 결과를 다시 sort reverse 로 뒤집음)

    global pre_val, stream_df2

    # 38일선 빼기 100일선 값을 저장하는 칼럼 생성
    # stream_df2 = stream_Query1 => 성공
    stream_Query1 = stream_Query1.withColumn("Diff", when(col("EMA100").isNotNull(), col("EMA38") - col("EMA100")).otherwise(lit(None)))
    
    # EMA_Comp 함수를 통해 Buy&Sell 시점 표시하는 칼럼 생성
    '''udf_EMA_Comp = udf(EMA_Comp, StringType())
    stream_Query1 = stream_Query1.withColumn("BuyOrSell", udf_EMA_Comp(stream_Query1["Diff"]))
    
    stream_Query1 = stream_Query1.select("symbol", "EMA38", "EMA100", "lastWindowTime", "Sectype", "Diff", "BuyOrSell")'''
    
    return stream_Query1

## results of Query 1

In [ ]:
def send_to_message_query1(lookup_symbols):
  global prevEMA
  #print("send_to_message_query1")
  result_Q1 = ch.ResultQ1()
  list_indicators = list()

  for s in lookup_symbols: # search for symbols required
    if s in prevEMA.index: # if prevEMA tables has information about previous EMAs.
        row = prevEMA.loc[s]
        indicator = ch.Indicator(symbol = row['symbol'], \
                                ema_38 = row['EMA38'], \
                                ema_100 =  row['EMA100'] )
        
    else: # else , send with 0s
        indicator = ch.Indicator(symbol = s, \
                                ema_38 = 0, \
                                ema_100 =  0 )
    list_indicators.append(indicator)
  
  # print("RESULT Q1")
  # print(list_indicators)
  result_Q1.indicators.extend(list_indicators)
  # print(result_Q1.indicators)
  
  return result_Q1.indicators

## results of Query 2

In [ ]:
def send_to_message_query2(lookup_symbols):
    global crossover
    crossover = crossover[crossover['BuyOrSell'] != 'Stay']
    tail_crossover = crossover.groupby(['symbol']).tail(3)
    # print("tail Extract")
    tail_crossover =tail_crossover.reset_index(drop=True)
    # print(tail_crossover)
    result_Q2 = ch.ResultQ2()
    list_crossover = list()
    for i in range(len(tail_crossover)): # search for symbols required
      cross_over_event = ch.CrossoverEvent()
      row = tail_crossover.loc[i,:]
      # print(row)
      cross_over_event.symbol = row["symbol"]

      if row["BuyOrSell"] == "Buy":
        cross_over_event.signal_type = ch.CrossoverEvent.SignalType.Buy
      elif row["BuyOrSell"] == "Sell":
        cross_over_event.signal_type = ch.CrossoverEvent.SignalType.Sell

      if row["Sectype"] == "E" or row["Sectype"] == "0":
        cross_over_event.security_type = ch.SecurityType.Equity
      else:
        cross_over_event.security_type = ch.SecurityType.Index
      
      timestamp = Timestamp()
      millis_time = row["lastWindowTime"].timestamp() * 1000
      secondes_time = time.mktime(row["lastWindowTime"].timetuple())
      cross_over_event.ts.seconds = int(secondes_time)
      list_crossover.append(cross_over_event)

    result_Q2.crossover_events.extend(list_crossover)
    # print(result_Q2.crossover_events)
    return result_Q2.crossover_events

## Streamed file processing
This funtions saves the stream of stock market data as csv files.

In [ ]:
def make_CSV_file(batch_no, events):
    fieldnames = ["id", "symbol", "Sectype", "lasttradeprice", "lastTrade"]
    event_list_dict = []
    for i in range(len(events)):
            temp_dict = {}
            temp_dict['id'] = i
            temp_dict['symbol'] = events[i].symbol
            temp_dict['Sectype'] = events[i].security_type
            temp_dict['lasttradeprice'] = events[i].last_trade_price
            temp_dict['lastTrade'] = datetime.fromtimestamp(events[i].last_trade.seconds).strftime("%Y-%m-%d %H:%M:%S")
            # temp_time = temp_time.strftime("%m/%d/%Y %H:%M:%S")
            # temp_time = str(temp_time)
            # temp_time_split = temp_time.split(" ")
            # temp_dict['Trading time'] = temp_time_split[1]
            # temp_dict['Trading date'] = temp_time_split[0]
            event_list_dict.append(temp_dict)

    # print(event_list_dict)
    with open(folderDir + f'/stock_stream-{batch_no}.csv', 'w+', encoding='UTF8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(event_list_dict)

## Main Program

setup global variables

In [ ]:
global spark, sc
global calcEMA, detectCrossover
global prevEMA
# global initialStateRDD

### initialize spark

In [ ]:
# start Spark application and get Spark session, logger and config
# spark application 시작
spark = SparkSession.builder.master("local[*]") \
        .config("spark.executor.instances", "2") \
        .config("spark.executor.cores", "2") \
        .config('spark.ui.port', '4050') \
        .config("spark.driver.memory", "20g")\
        .config("spark.executor.memory", "20g")\
        .config("spark.dynamicAllocation.enabled", "true") \
        .appName("Spark for DEBS 2020 GC").getOrCreate()
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")

sc = spark.sparkContext
# prevEMA = spark.sparkContext.broadcast({})

# 전역변수 선언. spark 구조적으로 공유가 안되는 것 같음
# 예상되는 원인 : spark의 udf가 Dataframe의 각 행마다 병렬적으로 처리되어 전역변수의 초기값이 한꺼번에 적용된다.
#               혹은 각 워커노드에 각 행이 따로 실행되어 전역변수의 갱신이 공유되지 않는 상황이 발생한다.
folderDir = "/content/drive/MyDrive/stream_practice/"

### Define streaming input sources

In [ ]:
# execute ETL pipeline
# structured streaming 형식으로 csv 파일 읽기 시작
stream = read_stream(folderDir)

In [ ]:
# # initialStateRDD = sc.parallelize([(u'pair_msgs', '{"symbol": "none", "ema38": 0, "ema100": 0 }')])


# # RDD with initial state (key, value) pairs
# # initialStateRDD = sc.parallelize([(u'pair_msgs', '{"url_id":"none"}')])

# # create empty dataframe
# schema = StructType([
#         StructField('symbol', StringType(), True),
#         # StructField('lastWindowTime', TimestampType(), True) ,
#         StructField('pEMA38', DoubleType(), True) ,
#         StructField('pEMA100', DoubleType(), True)
# ])
# emptyRDD = spark.sparkContext.emptyRDD()
# prevEMA = spark.createDataFrame(emptyRDD, schema)
# prevEMA.createOrReplaceGlobalTempView("prevEMA")

In [ ]:
prevEMA = pd.DataFrame({'symbol': pd.Series(dtype='str'),
                   'EMA38': pd.Series(dtype='float'),
                   'EMA100': pd.Series(dtype='float')})

prevEMA.set_index('symbol')

,EMA38,EMA100
symbol,,


In [ ]:
prevEMA = prevEMA.iloc[0:0]
prevEMA

,symbol,EMA38,EMA100


In [ ]:
prevEMA

,symbol,EMA38,EMA100


In [ ]:
crossover = pd.DataFrame({'symbol': pd.Series(dtype='str'),
                   'lastWindowTime': pd.Series(dtype='datetime64[ns]'),
                   'EMA38': pd.Series(dtype='float'),
                   'EMA100': pd.Series(dtype='float'),
                   'Sectype': pd.Series(dtype='str'),
                   'BuyOrSell': pd.Series(dtype='str')})

# crossover.set_index('symbol')

In [ ]:
prevCrossover = pd.DataFrame({'symbol': pd.Series(dtype='str'),
                   'Diff': pd.Series(dtype='float')})

prevCrossover.set_index('symbol')

,Diff
symbol,


In [ ]:
crossover = crossover.iloc[0:0]
crossover

,symbol,lastWindowTime,EMA38,EMA100,Sectype,BuyOrSell


### 2. Transform streaming data and define output sink&mode
queries 1 and 2

In [ ]:
calcEMA = Query1(stream)
# 3. lookup_symbol filter
"""if lookup_symbol_list != "":
    stream_Query1 = filter_symbol(writer1, lookup_symbol_list)"""

writer1 =  calcEMA.writeStream.format("memory") \
    .outputMode("complete") \
    .option("truncate", False) \
    .queryName("EMA") # table

In [ ]:
detectCrossover = Query2(calcEMA)

writer2 = detectCrossover.writeStream.format("memory") \
    .outputMode("complete") \
    .option("truncate", False) \
    .queryName("Crossover")

### Start the queries

In [ ]:
streamingQuery1 = writer1.start()
streamingQuery2 = writer2.start()

In [ ]:
# # spark.sql("select * from EMA").show()
# df2 = spark.sql("select distinct symbol from EMA").toPandas()
# df2

In [ ]:
#쿼리1 데이터 로드확인
# temp_df = stream_df1.select("symbol", "lasttradeprice","EMA38", "EMA100", "lastWindowTime", "Sectype") #tranformation
# temp_df.writeStream.format("memory").outputMode("complete").queryName("datastream").start() # register query

In [ ]:
# spark.sql("select * from datastream").show()
# df1 = spark.sql("select * from datastream").toPandas()

In [ ]:
prevEMA

,symbol,EMA38,EMA100


In [ ]:
def update_prevEMA(df_a):
  global prevEMA, crossover
  for idx, row in tqdm(df_a.iterrows(), total = df_a.shape[0], disable=True):
    symbol = row['symbol']
  #  print(gPrevEMA.loc[symbol])
    if symbol in prevEMA.index:
      # print(f"exist symbol {symbol}")
      existing_row = prevEMA.loc[symbol]
      '''print("pEMA38: ",  existing_row['EMA38'])
      print("pEMA100: ",  existing_row['EMA100'])'''
      # print("symbol: ", symbol)
      new_EMA38 = calcEMA_func(symbol, row['lasttradeprice'], existing_row['EMA38'], 38 )
      new_EMA100 = calcEMA_func(symbol, row['lasttradeprice'], existing_row['EMA100'], 100 )
      '''print(f"new EMA {symbol}")
      print("nEMA38: ",  new_EMA38)
      print("nEMA100: ",  new_EMA100)'''
      prevEMA.loc[symbol] = { 'symbol': symbol, 'EMA38': new_EMA38, 'EMA100': new_EMA100}
      # print("update prevEMA Table")
      # print(prevEMA.loc[symbol])
      # print( new_EMA38, new_EMA100 )

      new_row = { 'symbol': symbol, 'lastWindowTime': row['lastWindowTime'],'EMA38': new_EMA38, 'EMA100': new_EMA100, 'Sectype': row['Sectype'], 'BuyOrSell' : 'Stay' }
      crossover = crossover.append(pd.Series(new_row, index=crossover.columns, name=symbol))
      
      # print("----------------------------------------------------------- ")
    else: 
      # print(f"new symbol {symbol}")
      # new_row = pd.Series({'symbol': row['symbol'], 'EMA38': row['pEMA38'], 'EMA100': row['pEMA100']  })
      new_EMA38 = calcEMA_func(symbol, row['lasttradeprice'], 0.0, 38 )
      new_EMA100 = calcEMA_func(symbol, row['lasttradeprice'], 0.0, 100 )
      new_row = { 'symbol': symbol, 'lastWindowTime': row['lastWindowTime'],'EMA38': new_EMA38, 'EMA100': new_EMA100, 'Sectype': row['Sectype'], 'BuyOrSell' : 'Stay' }
      crossover = crossover.append(pd.Series(new_row, index=crossover.columns, name=symbol))

      new_row = { 'symbol': symbol, 'EMA38': new_EMA38, 'EMA100': new_EMA100}
      prevEMA = prevEMA.append(pd.Series(new_row, index=prevEMA.columns, name=symbol))
      # prevEMA.loc[symbol] = 
      # prevEMA = prevEMA.append(new_row)

    
  # print("completed.")
  # print(prevEMA)
  # return prevEMA 

In [ ]:
print(crossover)

Empty DataFrame
Columns: [symbol, lastWindowTime, EMA38, EMA100, Sectype, BuyOrSell]
Index: []


In [ ]:
def update_prevCrossover():
  global crossover, prevCrossover
  # print("============================================")
  #print('prevCrossover Table show')
  #print(prevCrossover)
  for idx, row in tqdm(crossover.iterrows(), total = crossover.shape[0], disable=True):
    symbol = row['symbol']
    if symbol not in prevCrossover.index:
      # new_row = pd.Series({'symbol': row['symbol'], 'EMA38': row['pEMA38'], 'EMA100': row['pEMA100']  })
      new_row = { 'symbol': symbol, 'Diff': row['EMA38'] - row['EMA100']}
      prevCrossover = prevCrossover.append(pd.Series(new_row, index=prevCrossover.columns, name=symbol))
    # print(prevCrossover)
    existing_row = prevCrossover.loc[symbol]
    BuyOrSell = EMA_Comp(row['EMA38']-row["EMA100"], existing_row['Diff'])
    # 값 갱신
    cond = (crossover.symbol == symbol) & (crossover.lastWindowTime == row['lastWindowTime'])
    crossover.loc[cond, ('BuyOrSell')] = BuyOrSell
    prevCrossover.loc[symbol] = { 'symbol': symbol, 'Diff':row['EMA38']-row["EMA100"]}
    # print(prevCrossover)

In [ ]:
def reset_dataframe(lookup_symbols):
  global prevEMA, crossover, prevCrossover
  for s in range(len(lookup_symbols)):
    spec = prevEMA['symbol'].str.contains(lookup_symbols[s]).index
    if lookup_symbols[s] in prevEMA.index:
      prevEMA.drop(spec,inplace = True)
    spec = crossover['symbol'].str.contains(lookup_symbols[s]).index
    if lookup_symbols[s] in crossover.index:
      crossover.drop(spec,inplace = True)
    spec = prevCrossover['symbol'].str.contains(lookup_symbols[s]).index
    if lookup_symbols[s] in prevCrossover.index:
      prevCrossover.drop(spec,inplace = True)

## Challenger system

In [ ]:
# %%timeit
# setting the option for GRPC
op = [('grpc.max_send_message_length', 100 * 1024 * 1024),
      ('grpc.max_receive_message_length', 1000 * 1024 * 1024)]
# global gPrevEMA

# Submit results with benchmark
with grpc.insecure_channel('challenge.msrg.in.tum.de:5023', options=op) as channel:
    stub = api.ChallengerStub(channel)
    # create_benchmark(stub)


    # Step 1 - Create a new Benchmark
    benchmarkconfiguration = ch.BenchmarkConfiguration(token= TOKEN_KEY,
                                                        benchmark_name="Dept. Computer Engineering, DongA Univ.",
                                                        # This name is used here: https://challenge.msrg.in.tum.de/benchmarks/
                                                        benchmark_type="test",
                                                        # Test or Evaluation, Evaluation will be available end of January. Test can be used to start implementing
                                                        queries=[ch.Query.Q1, ch.Query.Q2])
    benchmark = stub.createNewBenchmark(benchmarkconfiguration)

    stub.startBenchmark(benchmark)
    event_count = 0

    while True:
        batch = stub.nextBatch(benchmark)
        event_count = event_count + len(batch.events)
        #print(f"--- BATCH {batch.seq_id}: ID {benchmark.id} ")

        # batch.symbol로부터 받은 주식ID추출
        lookup_symbols = list(batch.lookup_symbols)
        # print(f"list for lookup: {lookup_symbols}")
        # reset_dataframe(lookup_symbols)
        # 1. nextBatch CSV파일로 저장
        make_CSV_file(batch.seq_id, batch.events)
        # reset_dataframe(lookup_symbols)
        # Symbol 별로 최근의 EMA 데이터만 유지
        lastTrades = spark.sql("select * from EMA").filter(col('symbol').isin(lookup_symbols)).toPandas()
        lastTrades.rename(columns = {'EMA38' : 'pEMA38'}, inplace = True)
        lastTrades.rename(columns = {'EMA100' : 'pEMA100'}, inplace = True)
        '''lastTrades = subset_df.groupBy(col("symbol")) \
                  .agg(last("lastWindowTime").alias("max_time_unix"), last("EMA38").alias("pEMA38"), last("lasttradeprice").alias("lasttradeprice"), last("EMA100").alias("pEMA100"), last("Sectype").alias("Sectype")  ) \
                  .toPandas() '''

        # print(lastTrades)
        update_prevEMA(lastTrades)
        #print("=>> current status of prev EMA tables")
        # print(prevEMA)
        
        # for idx, row in tqdm(lastTrades.iterrows(), total = lastTrades.shape[0], disable=True):
        # lookup_symbol_list로 받은 주식ID에 해당하는 row만 필터
        # filtered_df = spark.sql("select * from EMA").filter(col('symbol').isin(lookup_symbols)).toPandas()

        # spark.sql("select * from EMA").show()
        resultQ1 = ch.ResultQ1(
            benchmark_id=benchmark.id,  # The id of the benchmark
            batch_seq_id=batch.seq_id,  # The sequence id of the batch
            indicators= send_to_message_query1(lookup_symbols))  # Query1을 계산하여 challenge 서버에 보낼 message 생성
        stub.resultQ1(resultQ1)  # send the result of query 1 back
        
        # crossover_df = spark.sql("select * from Crossover")
        # Crossover 테이블 lookup_symbol_list에 맞게 필터
        '''filter_symbol_df2 = spark.sql("select * from Crossover").filter(col('symbol').isin(lookup_symbols)).toPandas()
        # print(filter_symbol_df2)

        # Stay이벤트가 아닌 것만 추출
        filter_stay = filter_symbol_df2[filter_symbol_df2['BuyOrSell'] != 'Stay']'''
        
        # 각 symbol별로 마지막 3개의 크로스오버 열 추출
        # crossover_df_tail = filter_stay.groupby(['symbol']).tail(3)
        # 구독symbol(lookup_symbol_list) 마다 마지막 3개의 crossover 추출
        update_prevCrossover()
        # 쿼리2 진행 결과 메세지 보내기
        resultQ2 = ch.ResultQ2(
            benchmark_id=benchmark.id,  # The id of the benchmark
            batch_seq_id=batch.seq_id,  # The sequence id of the batch
            crossover_events= send_to_message_query2(lookup_symbols)) # 마지막 3개 크로스오버를 계산하여 challenge 서버에 보낼 message 생성
        
        stub.resultQ2(resultQ2)  # submit the results of Q2
        # prevEMA, Crossover 테이블 리셋
        reset_dataframe(lookup_symbols)
        # Step 4 - once the last event is received, stop the clock
        # See the statistics within ~5min here: https://challenge.msrg.in.tum.de/benchmarks/
        if batch.last:
            print(f"received last batch, total batches: {event_count}")
            stub.endBenchmark(benchmark)
            break
    

completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.
completed.


KeyboardInterrupt: ignored

In [ ]:
print(f"Completed.")
spark.stop()

# Spark UI

In [ ]:
!pip install pyngrok
from pyngrok import ngrok

In [ ]:
NGROK_AUTH_TOKEN = "20Ayd0ezBx9EMWrfVGhPdif1skf_2h2MfcootBnk2WTBJf6GQ"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [ ]:
public_url = ngrok.connect(4050, "http")
public_url

await terminiation

In [ ]:
streamingQuery1.awaitTermination()
streamingQuery2.awaitTermination()

spark.stop()